# Rashba coupling

Construct a one-dimensional Hubbard model with the following non-interacting Hamiltonian, here expressed in $k$-space:
$$
    H_0 = \sum_k \mathcal{H}_k  ~~~~~ \mathcal{H}_k = -2t\cos k\;c^\dagger_k c_k - 2m \sin k\;c^\dagger_k \sigma_x c_k
$$
where $c_k$ is a two-component annihilation operator in spin space : $(c_{k,\uparrow}, c_{k,\downarrow})$ and $\sigma_x$ is a Pauli matrix.
Plot the non-interacting $\left(U=0\right)$ dispersion of this model as a function of $k$ in the interval $[-\pi,\pi]$ for $t=1$ and $m=0.2$ and check that it agrees with your analytical computation.

In [ ]:
import pyqcm
import numpy as np
import matplotlib.pyplot as plt

# declare a cluster model of 4 sites, named 'clus'
CM = pyqcm.cluster_model( 4)

# define a physical cluster based on that model, with base position (0,0,0) and site positions
clus = pyqcm.cluster(CM, ((0,0,0), (1,0,0), (2,0,0),(3,0,0))) 

# define a lattice model named '1D_4' made of the cluster(s) clus and superlattice vector (4,0,0)
model = pyqcm.lattice_model('1D_4', clus, ((4,0,0),))

# define a few operators in this model
model.interaction_operator('U')
model.hopping_operator('t', (1,0,0), -1)
# Defining the one-body Rashba coupling term
# tau=2 ---> imaginary contribution, ensures sin(k), sigma=1 ---> spin flip
model.hopping_operator("m", (1,0,0), -1, tau=2, sigma=1) 

In [ ]:
# Note that spin is not conserved due to the Rashba term
model.set_target_sectors('R0:N4') 
model.set_parameters("""
    t=1
    m=0.4
""")

In [ ]:
# Plotting the spectral function
I = pyqcm.model_instance(model)
I.spectral_function(wmax=2.5, path="line", nk=64, orb=1)

### Interpretation

As we can see, the addition of the Rashba coupling term to the kinetic hamilonian creates two energy levels instead of one.

### "Manual" energy eigenvalues

First, consider the following definition for the hamiltonian with Rashba coupling:
$$
    H_0 = \sum_{k}\mathcal{H}_k ~~,~~ \mathcal{H}_k = -2t\operatorname{cos}(k)c_k^\dagger c_k -2t\operatorname{sin}(k)c_k^\dagger\sigma_x c_k~~,~~ 
    c_k\equiv \begin{pmatrix}
        c_{k\uparrow} \\
        c_{k\downarrow}
    \end{pmatrix} \Rightarrow 
    c_k^\dagger = \begin{pmatrix}
        c_{k\uparrow}^\dagger &
        c_{k\downarrow}^\dagger
    \end{pmatrix}
$$
With these definitions, 
$$
    \mathcal{H}_k = -2t\operatorname{cos}(k)\left[ c_{k\uparrow}^\dagger c_{k\uparrow} + c_{k\downarrow}^\dagger c_{k\downarrow} \right] -2t\operatorname{sin}(k)\left[ c_{k\uparrow}^\dagger c_{k\downarrow} + c_{k\downarrow}^\dagger c_{k\uparrow} \right]
$$
Or more interestingly,
$$
    \mathcal{H}_k = c_k^\dagger
    \underbrace{\begin{pmatrix}
        -2t\operatorname{cos}(k) & -2m\operatorname{sin}(k) \\
        -2m\operatorname{sin}(k) & -2t\operatorname{cos}(k) 
    \end{pmatrix}}_{M}
    c_k
$$
Finding the eigenvalues of $M$ is trivial:
$$
    \varepsilon(k) = -2t\operatorname{cos}(k) \pm 2m\operatorname{sin}(k) = 2\left[ -t\operatorname{cos}(k) \pm m\operatorname{sin}(k) \right]
$$
Plotting these should yield the same result as the numerical computations above.


In [ ]:
# creating a grid of wavevectors in the Brillouin zone
k_grid = np.linspace(-np.pi, np.pi, 150)

t=1 # value of the hopping operator
m=0.2 # value of the coupling term


# Defining the energy eigenvalues found by hand
def eig_1(k, t, m):
    return -2*(t*np.cos(k) + m*np.sin(k))

def eig_2(k, t, m):
    return -2*(t*np.cos(k) - m*np.sin(k))

In [ ]:
# Plotting the analytic dispersion curve
fig, ax = plt.subplots()

ax.plot(k_grid, eig_1(k_grid, t, m), label="$\epsilon_1(k)$")
ax.plot(k_grid, eig_2(k_grid, t, m), label="$\epsilon_2(k)$")

ax.set_xlabel("k")
ax.set_ylabel("$\epsilon(k)$")

ax.set_xticks((-np.pi,0,np.pi))
ax.set_xticklabels(("$-\pi$","$0$","$-\pi$"))

ax.legend()

fig.show()